# งานที่ 2a: สรุปข้อความด้วยไฟล์ขนาดเล็กด้วย Titan Text Premier


ในโน้ตบุ๊กนี้ คุณได้นำข้อความขนาดเล็กเข้าสู่ Amazon Bedrock API โดยตรง (โดยใช้โมเดล Titan Text) และสั่งให้สรุปข้อความอินพุต คุณสามารถใช้วิธีการนี้เพื่อสรุปการถอดเสียงการโทร ถอดความประชุม หนังสือ บทความ โพสต์บล็อก และเนื้อหาที่เกี่ยวข้องอื่นๆ เมื่อความยาวข้อความอินพุตอยู่ภายในขอบเขตขนาดบริบทของโมเดล

## งานที่ 2a.1: การตั้งค่าสภาพแวดล้อม

ในงานนี้ ให้ตั้งค่าสภาพแวดล้อมของคุณ

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3
import botocore

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))

bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))



## งานที่ 2a.2: การเขียนพรอมต์คำสั่งพร้อมข้อความที่จะสรุป

ในงานนี้ ให้คุณใช้ข้อความสั้นๆ ที่มีโทเค็นน้อยกว่าความยาวสูงสุดที่รองรับโดยโมเดลพื้นฐาน ในฐานะตัวอย่างข้อความอินพุตสำหรับแล็บนี้ ให้คุณใช้ย่อหน้าจาก [โพสต์บล็อก AWS] (https://aws.amazon.com/jp/blogs/machine-learning/announcing-new-tools-for-building-with-generative-ai-on-aws/) ที่ประกาศใน Amazon Bedrock

พรอมต์คำสั่งเริ่มต้นด้วยคำแนะนำ `กรุณาให้สรุปข้อความต่อไปนี้` 

In [ ]:
prompt_data = """

Please provide a summary of the following text:

AWS took all of that feedback from customers, and today we are excited to announce Amazon Bedrock, \
a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API. \
Bedrock is the easiest way for customers to build and scale generative AI-based applications using FMs, \
democratizing access for all builders. Bedrock will offer the ability to access a range of powerful FMs \
for text and images—including Amazons Titan FMs, which consist of two new LLMs we’re also announcing \
today—through a scalable, reliable, and secure AWS managed service. With Bedrock’s serverless experience, \
customers can easily find the right model for what they’re trying to get done, get started quickly, privately \
customize FMs with their own data, and easily integrate and deploy them into their applications using the AWS \
tools and capabilities they are familiar with, without having to manage any infrastructure (including integrations \
with Amazon SageMaker ML features like Experiments to test different models and Pipelines to manage their FMs at scale).
"""

## งานที่ 2a.3: การสร้าง Request Body ด้วยพารามิเตอร์พรอมต์คำสั่งและพารามิเตอร์เชิงอนุมาน 

ในงานนี้ ให้คุณสร้าง Request Body ด้วยพรอมต์คำสั่งและพารามิเตอร์เชิงอนุมานข้างต้น

In [ ]:
# request body
body = json.dumps({
    "inputText": prompt_data, 
    "textGenerationConfig":{
        "maxTokenCount":2048,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
        }
    }) 

## งานที่ 2a.4: เรียกโมเดลพื้นฐานผ่าน Boto3

ในงานนี้ ให้คุณส่งคำขอ API ไปยัง Amazon Bedrock โดยระบุพารามิเตอร์คำขอ: `modelId`, `accept` และ `contentType` ตามพรอมต์คำสั่งที่ให้ไว้ โมเดลพื้นฐานใน Amazon Bedrock จะสรุปข้อความอินพุต

### สร้างเอาต์พุตให้เรียบร้อย

โดยค่าเริ่มต้น บริการ Amazon Bedrock จะสร้างสรุปทั้งหมดสำหรับพรอมต์คำสั่งที่กำหนดในเอาต์พุตเดียว สิ่งนี้อาจช้าหากเอาต์พุตโมเดลมีโทเค็นจำนวนมาก 

In [ ]:
#model configuration and invoke the model
modelId = 'amazon.titan-text-premier-v1:0' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"

try:

    response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    outputText = response_body.get('results')[0].get('outputText')

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

print(outputText)

### สตรีมการสร้างเอาต์พุต

ถัดไป ให้คุณสำรวจวิธีใช้ invoke_model_with_response_stream API ของ Amazon Bedrock เพื่อสตรีมเอาต์พุตโมเดลเพื่อให้ผู้ใช้สามารถใช้เอาต์พุตขณะที่สร้างขึ้นได้ แทนที่จะสร้างเอาต์พุตเต็มรูปแบบในครั้งเดียว API นี้จะส่งค่า ResponseStream ที่ส่งเอาต์พุตขนาดเล็กจากโมเดลขณะที่ผลิตขึ้น คุณสามารถแสดงเอาต์พุตสตรีมเหล่านี้ในมุมมองที่ใช้งานได้อย่างต่อเนื่อง

In [ ]:
#invoke model with response stream
modelId = 'amazon.titan-text-premier-v1:0'
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = list(stream)
output

In [ ]:
from IPython.display import display_markdown,Markdown,clear_output

In [ ]:
modelId = 'amazon.titan-text-premier-v1:0'
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['outputText']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

ตอนนี้คุณได้ทดลองใช้ boto3 SDK เพื่อเข้าถึง Amazon Bedrock API แล้ว SDK นี้ให้การเข้าถึงแบบโปรแกรมขั้นพื้นฐานสำหรับความสามารถของ Bedrock ด้วยการใช้ API นี้คุณสามารถใช้กรณีการใช้งานได้สองกรณี คือ 1) สร้างบทสรุปข้อความทั้งหมดของเนื้อหาข่าว AWS พร้อมกัน และ 2) สตรีมเอาต์พุตสรุปเป็นชิ้นๆ สำหรับการประมวลผลเพิ่มขึ้น

### ลองด้วยตัวเอง
- เปลี่ยนพรอมต์คำสั่งตามฐานการใช้งานเฉพาะของคุณและประเมินเอาต์พุตของโมเดลต่างๆ
- เล่นด้วยความยาวโทเค็นเพื่อทำความเข้าใจเวลาแฝงและการตอบกลับของบริการ
- ใช้หลักการวิศวกรรมการโต้ตอบที่แตกต่างกันเพื่อให้ได้เอาต์พุตที่ดีขึ้น

### เก็บงาน

คุณทำโน้ตบุ๊กนี้เสร็จแล้ว หากต้องการย้ายไปยังส่วนถัดไปของแล็บ ให้ทำดังนี้

- ปิดไฟล์โน้ตบุ๊กนี้และไปต่อยัง **Task2b.ipynb**